In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
import csv
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
import kerastuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
import pandas as pd
import winsound

In [2]:
gpu_options= tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.5)
tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options= gpu_options))

In [3]:
i=0
YES_SOURCE_DIR = "facemask-dataset/data/with_mask/"
TRAINING_YES_DIR = "facemask-dataset/train/with_mask/"
TESTING_YES_DIR = "facemask-dataset/test/with_mask/"
NO_SOURCE_DIR = "facemask-dataset/data/without_mask/"
TRAINING_NO_DIR = "facemask-dataset/train/without_mask/"
TESTING_NO_DIR = "facemask-dataset/test/without_mask/"

In [4]:
def making_dataset(SOURCE):
    dataset = []
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    shuffled_set = random.sample(dataset, len(dataset))
    return shuffled_set


def split_data(shuffled_set, SOURCE, TRAINING, TESTING, SPLIT_SIZE, k, n_folds):
    l = len(shuffled_set) / n_folds

    test_set = shuffled_set[int((k-1)*l) : int(k*l)]
    for unitData in test_set:
        temp_test_set = SOURCE + unitData
        final_test_set = TESTING + unitData
        copyfile(temp_test_set, final_test_set)

    train_set = shuffled_set[0:int((k-1)*l)] + shuffled_set[int(k*l): len(shuffled_set)]
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    




In [5]:
def build_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(176, kernel_size = 5, activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),

    tf.keras.layers.Conv2D(192, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
     
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(160, activation='relu'),
    
    tf.keras.layers.Dropout(0.7),
    
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
    ])
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def evaluate_model(model, TRAINING_YES_DIR, TESTING_YES_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, i):
    name = "lg-" + str(i+1) + "-" + str(int(time.time()))
    logdir = "log/" + name
    tb = TensorBoard(log_dir = logdir, histogram_freq=1, write_images=True, update_freq='epoch', embeddings_freq=5)
    TRAINING_DIR = "facemask-dataset/train-" + str(i+1)
    train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

    train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=32, 
                                                    target_size=(150, 150))
    VALIDATION_DIR = "facemask-dataset/test-" + str(i+1)
    validation_datagen = ImageDataGenerator(rescale=1.0/255)

    validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=32, 
                                                         target_size=(150, 150))
    model.load_weights('model.h5')
    checkpoint_path = 'best_model-'+ str(i+1)
    mc = ModelCheckpoint(checkpoint_path,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, restore_best_weights=True)
    
    history = model.fit(train_generator,
                        epochs = 500,
                        validation_data = validation_generator,
                        callbacks = [es, mc, tb], verbose=1)
    
    model = load_model(checkpoint_path)
    
    test_acc = model.evaluate(validation_generator, verbose = 1)
    '''
    #plotting loss 
    loss_train = history.history['loss']
    loss_val = history.history['val_loss']
    epochs = range(1, len(history.history['loss']) + 1)
    plt.plot(epochs, loss_train, 'g', label="training loss")
    plt.plot(epochs, loss_val, 'b', label="validation loss")
    plt.title("training and validation loss")
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()
    plt.show()

    #plotting accuracy
    acc_train = history.history['accuracy']
    acc_val = history.history['val_accuracy']
    epochs = range(1, len(history.history['accuracy']) + 1)
    plt.plot(epochs, acc_train, 'g', label="training acc")
    plt.plot(epochs, acc_val, 'b', label="validation acc")
    plt.title("training and validation acc")
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.legend()
    plt.show() 
    '''
    model.save_weights('model.h5')
    return model, test_acc

In [12]:
n_folds = 5
cv_scores, model_history = list(), list()
YES_SOURCE_DIR = "facemask-dataset/data/with_mask/"
NO_SOURCE_DIR = "facemask-dataset/data/without_mask/"
#YES_SET = making_dataset(YES_SOURCE_DIR)
#NO_SET = making_dataset(NO_SOURCE_DIR)
mod = build_model()
#mod.save_weights('model.h5')
for i in range(n_folds):
    
    TRAINING_YES_DIR = "facemask-dataset/train-" + str(5) + "/with_mask/"
    TESTING_YES_DIR = "facemask-dataset/test-" + str(5) + "/with_mask/"
    
    TRAINING_NO_DIR = "facemask-dataset/train-" + str(5) + "/without_mask/"
    TESTING_NO_DIR = "facemask-dataset/test-" + str(i+1) + "/without_mask/"
    # split data
    
    split_size = 0.8
    split_data(YES_SET, YES_SOURCE_DIR, TRAINING_YES_DIR, TESTING_YES_DIR, split_size, 5, n_folds)
    split_data(NO_SET ,NO_SOURCE_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, split_size, 5, n_folds)
    
    # evaluate model
    model, test_acc = evaluate_model(mod, TRAINING_YES_DIR, TESTING_YES_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, i)
    print("**********************************************************************************************")
    print('>%.3f' % test_acc[1])
    print("##############################################################################################")
    cv_scores.append(test_acc[1])
    model_history.append(model)
    
print('Estimated Accuracy: %.3f with standard deviaton: %.3f' % (np.mean(cv_scores), np.std(cv_scores)))
print("##############################################################################################")

Found 2006 images belonging to 2 classes.
Found 802 images belonging to 2 classes.
Epoch 1/500


KeyboardInterrupt: 

In [8]:
max = cv_scores.index(np.max(cv_scores))
checkpoint = 'best_model-'+ str(max+1)
mod = load_model(checkpoint)
mod.save('best_model')

INFO:tensorflow:Assets written to: best_model\assets


In [9]:
TEST_DIR = "facemask-dataset/val"
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                         batch_size=306,
                                                         class_mode="binary",
                                                         target_size=(150, 150))
testx, testy = next(test_generator)
print(test_generator.class_indices)
predIdxs = mod.predict(testx, batch_size=306)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testy, predIdxs,target_names=['without_mask', 'mask']))
print("##############################################################################################")
print("Best accuracy: " + str(cv_scores[max]))

Found 306 images belonging to 2 classes.
{'with_mask': 0, 'without_mask': 1}
              precision    recall  f1-score   support

without_mask       0.94      0.95      0.95       153
        mask       0.95      0.94      0.95       153

    accuracy                           0.95       306
   macro avg       0.95      0.95      0.95       306
weighted avg       0.95      0.95      0.95       306

##############################################################################################
Best accuracy: 0.9452736377716064


In [10]:
print(cv_scores)
df = pd.DataFrame(cv_scores)
df.to_csv('scores.csv', index=False, header=False)

[0.9399999976158142, 0.927860677242279, 0.9449999928474426, 0.9452736377716064]


In [13]:
labels_dict={0:'without_mask', 1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}
j = 0
counter = 0
size = 4
webcam = cv2.VideoCapture(0) #Use camera 0
#address = "https://192.168.43.1:8080/video"
#webcam.open(address)
checkpoint = 'best_model'
model = load_model(checkpoint)
# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    fps = webcam.get(cv2.CAP_PROP_FPS)
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror
    
    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]

        if label == 1:
            j+=1
        elif label == 0:
            j=0
            freq = 2500
            dur = 100
            winsound.Beep(freq, dur)

        if j>=30:
            j=0
            counter+=1

        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    # Show the image
    f = "FPS: "+ str(fps)
    cv2.putText(im, f, (30, 30),cv2.FONT_HERSHEY_SIMPLEX,1,(225,255,225),2)
    textsize = cv2.getTextSize(str(counter), cv2.FONT_HERSHEY_SIMPLEX, 6, 10)[0]
    xx = int((im.shape[1] - textsize[0]) / 2)
    yy = int((im.shape[0] - textsize[1]) / 2)
    cv2.putText(im, str(counter), (xx, yy + 100),cv2.FONT_HERSHEY_SIMPLEX,6,(0,255,0),10)
    cv2.imshow('LIVE', im)
    # if q key is press then break out of the loop 
    if cv2.waitKey(10) & 0xFF == ord("q"): #The q key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()